# 相机校准

### 导入头文件

In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import os

sys.path.append("../dofbot_garbage_yolov5/utils")
import Arm_Lib
import cv2 as cv
import threading
from time import sleep
from dofbot_config import ArmCalibration, read_XYT, write_XYT
import ipywidgets as widgets
from IPython.display import display
from garbage_identify import GarbageIdentify

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


### 创建实例,初始化参数

In [2]:
# 创建获取目标实例
target = GarbageIdentify(test_mode=True)
# 创建相机标定实例
calibration = ArmCalibration()
# 创建机械臂驱动实例
arm = Arm_Lib.Arm_Device()
# 初始化一些参数
num=0
# 初始化标定方框边点
dp = []
# 初始化抓取信息
msg = {}
# 初始化1,2舵机角度值
xy = [90, 135]
# 初始化二值图阈值
threshold = 140
# 初始化模式
model = "General"
# XYT参数路径
pkg_parent_dir = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
pkg_dir = os.path.join(pkg_parent_dir, "dofbot_garbage_yolov5")
cfg_dir = os.path.join(pkg_dir, "config")
XYT_cfg_dir = os.path.join(cfg_dir, "XYT_config.txt")
data_idx_cfg_dir = os.path.join(cfg_dir, "data_collect_idx.txt")
dp_cfg_dir = os.path.join(cfg_dir, "dp.bin")
XYT_path = XYT_cfg_dir
try: 
    xy, thresh = read_XYT(XYT_path)
    print("read xy is:", xy)
except Exception: 
    print("Read XYT_config Error !!!")

y_offset is 0.016
x_offset is 0.01
finish init..
read xy is: [88, 131]


### 初始化机械臂位置

In [3]:
import Arm_Lib
arm = Arm_Lib.Arm_Device()
joints_0 = [xy[0], xy[1], 0, 0, 90, 30] 
arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建控件

In [4]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 调整滑杆
joint1_slider      = widgets.IntSlider(description='joint1 :'   ,    value=xy[0]     , min=70 , max=110, step=1, orientation='horizontal')
joint2_slider      = widgets.IntSlider(description='joint2 :'   ,    value=xy[1]     , min=50, max=155, step=1, orientation='horizontal') # REVISE
threshold_slider   = widgets.IntSlider(description='threshold :',    value=threshold , min=0  , max=255, step=1, orientation='horizontal')

# 进入标定模式
calibration_model  = widgets.Button(description='calibration_model',  button_style='primary', layout=button_layout)
calibration_ok     = widgets.Button(description='calibration_ok',     button_style='success', layout=button_layout)
calibration_cancel = widgets.Button(description='calibration_cancel', button_style='danger', layout=button_layout)

# 退出
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)

# 添加布局
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
garbage_identify = widgets.VBox(
    [joint1_slider, joint2_slider, threshold_slider, calibration_model, calibration_ok, calibration_cancel, exit_button],
    layout=widgets.Layout(align_self='center'));
controls_box = widgets.HBox([imgbox, garbage_identify], layout=widgets.Layout(align_self='center'))

### 标定回调

In [5]:
# 标定回调
def calibration_model_Callback(value):
    global model
    model = 'Calibration'
    with output: 
        print(model)
def calibration_OK_Callback(value):
    global model
    model = 'calibration_OK'
    with output: 
        print(model)
def calibration_cancel_Callback(value):
    global model
    model = 'calibration_Cancel'
    with output: 
        print(model)
        
calibration_model.on_click(calibration_model_Callback)
calibration_ok.on_click(calibration_OK_Callback)
calibration_cancel.on_click(calibration_cancel_Callback)

In [6]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: 
        print(model)
        
exit_button.on_click(exit_button_Callback)

### 主程序

In [7]:
def camera():
    global model, dp, msg, HSV_name
    # 打开摄像头
    capture = cv.VideoCapture(0)
    print("Cam has FPS:", capture.get(cv.CAP_PROP_FPS))
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            # 读取相机的每一帧
            _, img = capture.read()
            # 统一图像大小
            img = cv.resize(img, (640, 480))
            xy = [joint1_slider.value, joint2_slider.value]
            if model == 'Calibration':
                _, img = calibration.calibration_map(img, xy, threshold_slider.value)
            if model == 'calibration_OK':
                try: 
                    print("Try to write dp..")
                    write_XYT(XYT_path, xy, threshold_slider.value)
                    print("Finish Write dp..")
                except Exception: 
                    print("File XYT_config Error !!!")
                dp, img = calibration.calibration_map(img, xy, threshold_slider.value)
                print("dp is", dp)
                dp.tofile(dp_cfg_dir)
                print("saved bin..")
                model="General"
            if len(dp) != 0: 
                img = calibration.perspective_transform(dp, img)
            if model == 'calibration_Cancel':  
                dp = []
                msg= {}
                model="General"
                
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:capture.release()

### 启动

In [8]:
display(controls_box,output)
threading.Thread(target=camera, ).start()

Output()

Cam has FPS: 30.0
Try to write dp..
Finish Write dp..
dp is [[ 75  35]
 [ 14 452]
 [627 436]
 [540  22]]
saved bin..
